# Tornado Synthetic Dataset - 3D Steady
> [cgl datasets](https://cgl.ethz.ch/research/visualization/data.php)
> Steady means no time is involved

In [27]:
import numpy as np
import pyvista as pv
# np.set_printoptions(precision=3)

In [28]:
file = '..\\Data\\tornado3d.vti'
mesh = pv.read(file)
print(mesh)

UniformGrid (0x2578a426a60)
  N Cells:	2048383
  N Points:	2097152
  X Bounds:	-1.000e+01, 1.000e+01
  Y Bounds:	-1.000e+01, 1.000e+01
  Z Bounds:	-1.000e+01, 1.000e+01
  Dimensions:	128, 128, 128
  Spacing:	1.575e-01, 1.575e-01, 1.575e-01
  N Arrays:	3



In [29]:
w,v,u = mesh.point_data['w'], mesh.point_data['v'], mesh.point_data['u']

vectors = np.empty((mesh.n_points, 3))
vectors[:,0] = u
vectors[:,1] = v
vectors[:,2] = w

print(f'mesh dimensions : {mesh.dimensions}')
print(f'128 * 128 * 128 = {128*128*128}')
print(f'u shape : {u.shape}\nv shape : {v.shape}\nw shape : {w.shape}')
print(f'vectors shape : {vectors.shape}')

mesh dimensions : (128, 128, 128)
128 * 128 * 128 = 2097152
u shape : (2097152,)
v shape : (2097152,)
w shape : (2097152,)
vectors shape : (2097152, 3)


In [30]:
# https://docs.pyvista.org/api/core/_autosummary/pyvista.RectilinearGrid.streamlines.html

mesh['vectors'] = vectors

boundary = mesh.decimate_boundary().extract_all_edges()

# Sphere Source
# stream, src = mesh.streamlines(
#     'vectors', 
#     return_source=True,
#     terminal_speed=0,
#     n_points=10,
#     source_radius=1,
#     source_center=(0,0,9)
# )

# Line Source
stream, src = mesh.streamlines(
    'vectors', 
    return_source=True,
    n_points=100,
    pointa=(-8, -8, 10),
    pointb=(8, 8, 10),
    integration_direction='both',
    terminal_speed=-1,
    step_unit='l',
    initial_step_length=.01,
    max_steps=6000,
    min_step_length=.01
)

sargs=dict(vertical=True, title_font_size=16)
p = pv.Plotter()
p.add_mesh(stream.tube(radius=0.02), lighting=False, scalar_bar_args=sargs)
p.add_mesh(src)
p.add_mesh(boundary, color='grey', opacity=.25)
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [31]:
print(stream.point_data)

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : vectors
Active Vectors  : Normals
Active Texture  : None
Active Normals  : None
Contains arrays :
    u                       float32  (355167,)
    v                       float32  (355167,)
    w                       float32  (355167,)
    vectors                 float64  (355167, 3)          SCALARS
    IntegrationTime         float64  (355167,)
    Vorticity               float64  (355167, 3)
    Rotation                float64  (355167,)
    AngularVelocity         float64  (355167,)
    Normals                 float64  (355167, 3)          VECTORS


# First Sample of s=2

In [43]:
s = 2

mesh2 = pv.UniformGrid(
    dims=tuple([int(x/s) for x in list(mesh.dimensions)]),
    spacing=tuple([x*s for x in list(mesh.spacing)]),
    origin=mesh.origin,
)

print(f'mesh spacing : {mesh.spacing}')
print(f'mesh2 spacing : {mesh2.spacing}')

ws,vs,us = mesh.point_data['w'], mesh.point_data['v'], mesh.point_data['u']
us = us[::s]
vs = vs[::s]
ws = ws[::s]
vectorss = np.empty((int(mesh.n_points/s), 3))
vectorss[:,0] = us
vectorss[:,1] = vs
vectorss[:,2] = ws

print(f'mesh dimensions : {mesh2.dimensions}')
print(f'(64 * 64 * 64)  = {(64*64*64)}')
print(f'us shape : {us.shape}\nvs shape : {vs.shape}\nws shape : {ws.shape}')
print(f'vectorss shape : {vectorss.shape}')

mesh2['vectors'] = vectorss

mesh spacing : (0.15748031496, 0.15748031496, 0.15748031496)
mesh2 spacing : (0.31496062992, 0.31496062992, 0.31496062992)
mesh dimensions : (64, 64, 64)
(64 * 64 * 64)  = 262144
us shape : (1048576,)
vs shape : (1048576,)
ws shape : (1048576,)
vectorss shape : (1048576, 3)


ValueError: Number of scalars (3145728) must match either the number of points (262144) or the number of cells (250047).

In [ ]:
mesh['vectorss'] = vectorss

# Sphere Source
# stream, src = mesh.streamlines(
#     'vectorss', 
#     source=src,
#     return_source=True,
#     terminal_speed=0,
#     n_points=10,
#     source_radius=1,
#     source_center=(0,0,9)
# )

# Line Source
stream = mesh.streamlines(
    'vectorss', 
    source=src, 
    integration_direction='both',
    terminal_speed=-1,
    step_unit='l',
    initial_step_length=.01,
    max_steps=6000,
    min_step_length=.01
)

sargs=dict(vertical=True, title_font_size=16)
p = pv.Plotter()
p.add_mesh(stream.tube(radius=0.02), lighting=False, scalar_bar_args=sargs)
p.add_mesh(src)
p.add_mesh(boundary, color='grey', opacity=.25)
p.show()

ValueError: Number of scalars (3145728) must match either the number of points (2097152) or the number of cells (2048383).